In [ ]:
import time
import re
'''
Variables
description: 아뜰리에 요약정보
totalReviewPage: 전체 리뷰 페이지 수
reviews: 각 리뷰 데이터 -> reviewList: 리뷰 데이터 리스트
scores: 각 별점 데이터 -> scoreList: 별점 데이터 리스트
reviewPage: 가져올 리뷰 페이지 

Functions
getData: 페이지별 리뷰데이터
getKeywords: 각 아뜰리에별 테마키워드
'''

def getData(storeId,storeName, reviewPage) :
    scoreList = []
    reviewList = []    
    
    for i in range(0, reviewPage):
        searchURL = "https://store.naver.com/restaurants/detail?entry=pll&id="+storeId+"&query="+storeName+"&tab=receiptReview&tabPage="+str(i)
        driver.get(searchURL)
        
        reviews = driver.find_elements_by_class_name("review_txt")
        scores = driver.find_elements_by_class_name("score")
        
        for review in reviews:
            text = review.text
            only_BMP_pattern = re.compile("["u"\U00010000-\U0010FFFF""]+", flags=re.UNICODE)
            text = only_BMP_pattern.sub(r'', text)
            reviewList.append(text)
            
        for score in scores:
            scoreList.append(float(score.text)) #소수로 저장
            
    return reviewList,scoreList

def getKeywords(storeId,storeName):
    #keywordList = []
    strAll = ""
    description = ""
    
    searchURL = "https://store.naver.com/restaurants/detail?entry=pll&id="+storeId+"&query="+storeName+"&tab=main"
    driver.get(searchURL)
    keywords =  driver.find_elements_by_class_name("kwd")
    description = driver.find_elements_by_class_name("ellipsis_area")[0]
    description = (description.find_elements_by_tag_name("span")[0]).text
    
    for keyword in keywords:
        strAll += keyword.text
    
    return description,strAll


In [ ]:
#아뜰리에 기본 데이터 불러오기 (아뜰리에명, 아뜰리에고유아이디)
from openpyxl import load_workbook

storeList = []
storeIdList = []

load_wb = load_workbook("fran_data.xlsx", data_only=True)
load_ws = load_wb['프렌차이즈 제거_전국 아뜰리에']

for i in range(2, 6227):
    storeList.append(load_ws.cell(row=i,column=2).value)
    storeIdList.append(load_ws.cell(row=i,column=3).value)

In [ ]:
print(storeList)

['자매상회', '안녕케이크', '카페그레이나인', '맑은바람', '투썸진천블로썸캠퍼스', '스노잉네이처', '꿀벌의아침', '블루문', '피넛커피 광혜원점', '카페두촌리', '롯데델리', '카페오랑', '타르티', '매니아카페', '카페사계', '카페바들말', '찹쌀루 광혜원점', '코지', '카페도로시', '픽미 진천덕산점', '카페203 진천남산길점', '화계절', '카페온유', '카페181', '커피프라넬', '홍스타디저트카페', '카페모드니', '까페담', '허브나무', '카페노란코끼리', 'MY디저트', '단하나의풍경', '담실갤러리까페', '하쿠나마타타', '마린데크', '문화제과', '1994커피', '배스킨라빈스삼척남양점', '부산어묵 쏠비치삼척', '어나더', '버블엔젤', '102커피', '지재근베이커리', '청년마카롱', '삼척명품찹쌀꽈배기', '근덕삼척꽈배기', '카페꼼지락', '구공리', 'STAY', '바다페', '커피정원', '이사부사자공원카페', '대궐', '카페공감', '여유앤화랑', '커피블라우', '스마일찰진꽈배기', '스윗가든', '때론347', '눈가루앤클라츠', '오버더문', '19티 삼척남양점', '데일리', '마카올래', '엘마르', '밀카페', '카페로이', '황금찹쌀꽈배기', '씨홀스베이커리인아그곳', 'Hill760', '토스피아', '옹기종기', '세븐몽키스커피 삼척점', '그리고봄', '마루베이커리', '팀버', '오늘참예쁘다', '페로어페로 김천구미점', '양조장카페', '호두와나무', '파파브레드', '못앞에커피', '달콩냥냥', '김천 못난이찹쌀 꽈배기', '어때', '간판없는커피집', '혹닉마카롱', '봄봄혁신점', '무드벨', '331로스터리카페', '르카페시크릿', '클램', '카페398', '부띠끄카페제이', '카페우아', '카페다홍', '쭈롱베이커리', '행복한마카롱 김천점', '던킨도너츠한국도로공사본사점', '이즈브레드', '카페레드빈', '후크커피', '끌레르', '잇츠', '소블링', 

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
import openpyxl


path = "resource/chromedriver"
driver = webdriver.Chrome(path)
write_wb = openpyxl.Workbook()
write_ws = write_wb.active

write_ws['A1'] = '아뜰리에명'
write_ws['B1'] = '설명'
write_ws['C1'] = '키워드'
write_ws['D1'] = '리뷰'
write_ws['E1'] = '평점'
cnt = 0 #리뷰개수가 30개~100개를 가지는 아뜰리에 넘버링

'''
TestData (description, count)
패션5 : O/440개 (리뷰가 100개 이상으므로 나오면 안됨)
르쁘띠푸: O/70개
까페베네치아: X/17개
달스윗:X/43개
'''

#테스팅 데이터
#storeIdList = [12851539,13156166,33144418,1372437122]
#storeList = ['패션5','르쁘띠푸','까페베네치아','달스윗'] 
keywordList = []
reviewList = []
scoreList = []
#크롤링 횟수 조절
countNum = 2501

#2000~2501
for i in range(2000,countNum):
    name = ""
    description = ""
    keywords = ""
    searchURL = "https://store.naver.com/restaurants/detail?entry=pll&id="+str(storeIdList[i])+"&query="+storeList[i]+"&tab=receiptReview"
    ss = 0
    try:
        while True:
            try:
                driver.get(searchURL)
                ss +=1
            except:
                driver.refresh()
            if ss == 1:
                break
            else:
                break
        ss = 0
        time.sleep(3)
        totalReviewPage = int(driver.find_elements_by_xpath("//*[@id='panel03']/div/div[2]/span")[0].text)            
        reviewPage = 0
                
        #리뷰 개수가 30개~100개까지 인 것만 
        if(totalReviewPage<4):
            continue
        elif(totalReviewPage>11):
            reviewPage = 3
        else: 
            reviewPage = totalReviewPage
        
        
    #리뷰데이터와 평점 데이터 
        while True: 
            try: 
                reviewList,scoreList = getData(str(storeIdList[i]),storeList[i],reviewPage)
                ss+=1
                #아뜰리에 설명 및 테마키워드
            except:
                driver.refresh()
            if ss==1:
                break
            else:
                break
        ss=0
        
        while True: 
            try: 
                description, keywords = getKeywords(str(storeIdList[i]),storeList[i])
                ss +=1
            except:
                driver.refresh()
            if ss==1:
                break
            else:
                break
        ss=0    
        name = storeList[i]
        cnt +=1
        print(cnt)
        print(name)
        
        #엑셀파일 셀에 저장        
        for k in range(0,len(reviewList)):
            review = reviewList[k]
            score = scoreList[k]
            write_ws.append([name,description,keywords,review,score])    
    except:
        driver.refresh()
write_wb.save('storeInfo_1001.xlsx')
        

1
더로드101
2
쌍계명차
3
우주총동원
4
카페포레
5
하동스토리하우스카페
6
키친205
7
커피로드333
8
161커피스튜디오
9
가창옛날찐빵
10
읍천리
11
오르망
12
이흥용과자점 사직점
13
텐퍼센트커피 부산시청본점
14
보느파티쓰리
15
어나더미네스
16
caffe gather
17
구자윤과자점
18
파리바게트거제현대
19
아뜰리에마롱
20
사이먼커피
21
텐퍼센트커피 교대법원점
22
chic985
23
커피긱스
24
디저트시네마
25
오감베이커리
26
하이드벗
27
파리바게뜨연산로터리점
28
빵85
29
다온브레드
30
니드썸레스트
31
르봉뺑
32
리버레인
33
플로레
34
코미호미
35
나무아래오후
36
더스틸카페
37
꼬무네 카페
38
라틴정원
39
나무아래오후N
40
아트살롱가평
41
카페75
42
미스터박 베이커리카페
43
카페쁘라제르
44
스테이83
45
바이스테이션
46
라쿠나
47
제빵소덤 대성리점
48
홍종흔베이커리
49
배스킨라빈스산본중심가점
50
라붐커피 산본점
51
디마인커피로스터즈
52
숨맑은집
53
허니듀
54
손커피연구소 산본역점
55
부산케키
56
콩만한커피알만한케익
57
반월호수 온도
58
와플크림
59
신일룡의 호두파이
60
카페언타이
61
조셉파리
62
요요연연
63
이백
64
커피라끄
65
카페피노
66
이프테이
67
여울목카페
68
커피단월
69
게으른악어
70
듀레베이커리 남산점
71
듀레베이커리
72
세상상회
73
활옥동굴카페
74
식스타임즈
75
샤인
76
배스킨라빈스충주국원점
77
모이까 금릉점
78
루암리
79
씨엘비베이커리
80
코롬방제과점
81
카페피어파이브
82
커피파나쉐 전망대점
83
카페닻
84
가비1935
85
쑥꿀레
86
커피창고로
87
배스킨라빈스목포상동점
88
그린하우스제과점
89
하루
90
미텐발트
91
애뜨락
92
미스티
93
베니어 베이커리카페
94
빛담
95
늘솜당
